# Embedded js_proxy showcase

[For the impatient, please scroll down to cell 4 and rotate the 3D teapot.]

The `js_proxy` framework provides a relatively simple
mechanism for using functionality implemented by javascript
libraries inside an IPython notebook without having
to write any javascript in many cases.

The `js_proxy` framework can be used in two modes:

- interactive widget mode, or
- embedded HTML mode.

This notebook showcases the use of `js_proxy` in embedded
mode.  Please see [the js_proxy examples](js_proxy%20example.ipynb)
for a more general discussion of the `js_proxy` programming
interface.

## Interactive widget js_proxy mode

The interactive widget mode is decribed in
[the js_proxy examples](js_proxy%20example.ipynb)
notebook.

As the name suggests interactive widgets
can communicate back and forth from the user/browser via
Javascript to the Python interpreter to implement
complex user interaction.

However, if user interactions that communicate
with the Python interpreter are not required
the interactive widget framework has the following
disadvantages

- visualizations created by the widgets do not
persists when the notebook is saved.

- the widget does not appear in "offline" viewing
such as via nbviewer.

## Embedded HTML js_proxy mode

The embedded mode allows visualizations and javascript-only
interactivity to persist when the notebook is saved
or when the notebook is viewed "offline".
The embedded mode does not support interactions that
communicate with the Python interpreter, however.

## Using `js_proxy` in embedded mode

To create a visualization using `js_proxy` in embedded mode
you typically need to do the following.

### Load required python imports

In [1]:
# allow import without install
import sys
if ".." not in sys.path:
    sys.path.append("..")

from jp_gene_viz import js_proxy

### Load any javascript libraries you will use

Here we load `SceneJS` which is library for 3D visualization and animations.

In [2]:
from IPython.display import display, HTML
load_Scenejs = """
<script src="http://scenejs.org/api/latest/scenejs.min.js"></script>
"""
display(HTML(load_Scenejs))

### Create the "widget" and load it's command buffer

In this case we transliterate one of the 
[the SceneJS examples](http://scenejs.org/examples/index.html#scenegraph_firstExample).
The notation used in the `js_proxy` context is similar to the Javascript provided in the
examples, except that the Python dictionary keys must be explicitly formatted as strings.

In [3]:
w = js_proxy.ProxyWidget()
elt = w.element()
window = w.window()
SceneJS= window.SceneJS
w(elt.append("""<canvas id="myCanvas002" width="400px" height="400px"></canvas>"""))
w(SceneJS.setConfigs({
        "pluginPath":"http://scenejs.org/api/latest/plugins"
        #"pluginPath":"http://localhost:4444/api/latest/plugins"
    }))
x, y, z, r, g, b = list("xyzrgb")
w.save("scene", SceneJS.createScene({
        "canvasId": "myCanvas002",
        "nodes":[

            #// Mouse-orbited camera,
            #// implemented by plugin at http://scenejs.org/api/latest/plugins/node/cameras/orbit.js
            {
                "type":"cameras/orbit",
                "yaw":40,
                "pitch":-20,
                "zoom":10,
                "zoomSensitivity":1.0,
                "eye":{ x:0, y:0, z:10 },
                "look":{ x:0, y:0, z:0 },

                "nodes":[

                    #// Blue material
                    {
                        "type":"material",
                        "color":{ r:0.3, g:0.3, b:1.0 },

                        "nodes":[

                            #// Teapot primitive,
                            #// implemented by plugin at http://scenejs.org/api/latest/plugins/node/geometry/teapot.js
                            {
                                "type":"geometry/teapot"
                            }
                        ]
                    }
                ]
            }
        ]
    }))
# DO NOT FLUSH OR DISPLAY!
#w.flush()
#display(w)

element
["scene"]

### Embed the widget as HTML

In this case we specify that the widget should make sure the `SceneJS`
global module has loaded before initializing.  This is important when the
notebook is loaded offline because some libraries (like SceneJS) load
asyncronously and may not be immediately available.

In [4]:
w.embed(True, await=["SceneJS"])

('id', 'jupyter_proxy_widget1')


### Enjoy the widget

In this case you should see a 3D teapot displayed above.
The scene is attached to "orbit" controls -- mousedown/drag
to rotate the scene.